In [19]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [20]:
start_date="2015-01-01"
end_date="2023-01-01"

In [21]:
data=yf.download('NVDA',start=start_date,end=end_date) 

[*********************100%%**********************]  1 of 1 completed


In [22]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,0.50325,0.50700,0.49525,0.50325,0.483218,113680000
2015-01-05,0.50325,0.50475,0.49250,0.49475,0.475057,197952000
2015-01-06,0.49550,0.49600,0.47925,0.47975,0.460654,197764000
2015-01-07,0.48325,0.48750,0.47700,0.47850,0.459453,321808000
2015-01-08,0.48400,0.49950,0.48375,0.49650,0.476737,283780000
...,...,...,...,...,...,...
2022-12-23,15.19600,15.33900,14.88300,15.20600,15.197296,349326000
2022-12-27,15.07400,15.10000,14.05600,14.12100,14.112916,464902000
2022-12-28,13.92700,14.26200,13.88400,14.03600,14.027965,351066000


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2014 entries, 2015-01-02 to 2022-12-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2014 non-null   float64
 1   High       2014 non-null   float64
 2   Low        2014 non-null   float64
 3   Close      2014 non-null   float64
 4   Adj Close  2014 non-null   float64
 5   Volume     2014 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 110.1 KB


In [24]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2.014000e+03
mean,7.979089,8.129079,7.818051,7.978563,7.946668,4.782126e+08
std,7.509996,7.670714,7.327442,7.501542,7.499611,2.696910e+08
min,0.481250,0.487500,0.473500,0.478500,0.459453,5.244800e+07
25%,2.471812,2.505625,2.440000,2.475750,2.442941,3.015300e+08
50%,5.241875,5.298750,5.128125,5.233375,5.193736,4.200140e+08
75%,13.141000,13.386625,12.954063,13.162187,13.147104,5.814090e+08
max,33.516998,34.646999,32.035999,33.375999,33.321644,3.692928e+09


In [25]:
data.isnull()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,False,False,False,False,False,False
2015-01-05,False,False,False,False,False,False
2015-01-06,False,False,False,False,False,False
2015-01-07,False,False,False,False,False,False
2015-01-08,False,False,False,False,False,False
...,...,...,...,...,...,...
2022-12-23,False,False,False,False,False,False
2022-12-27,False,False,False,False,False,False
2022-12-28,False,False,False,False,False,False


In [26]:
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [27]:
data1=data[['Close']].values

In [28]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(data1)

In [77]:
train_data=[]
target_data=[]

In [78]:
for a in range(60,len(scaled_data)):
    train_data.append(scaled_data[a-60:a,0])
    target_data.append(scaled_data[a,0])

In [79]:
train_data=np.array(train_data)
target_data=np.array(target_data)

In [80]:
train_data=np.reshape(train_data,(train_data.shape[0],train_data.shape[1],1))

In [81]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(train_data.shape[1],1)))
model.add(LSTM(units=50,return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

C:\Users\bolle\OneDrive\Documents\python\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [82]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [83]:
model.fit(train_data, target_data, epochs=20, batch_size=1)

Epoch 1/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 50s 24ms/step - loss: 0.0026
Epoch 2/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 82s 23ms/step - loss: 6.1141e-04
Epoch 3/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 46s 24ms/step - loss: 5.9946e-04
Epoch 4/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 46s 24ms/step - loss: 3.7639e-04
Epoch 5/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 47s 24ms/step - loss: 3.3941e-04
Epoch 6/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 46s 24ms/step - loss: 4.5648e-04
Epoch 7/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 46s 23ms/step - loss: 3.6778e-04
Epoch 8/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 69s 35ms/step - loss: 4.1496e-04
Epoch 9/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 63s 26ms/step - loss: 2.8484e-04
Epoch 10/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 48s 25ms/step - loss: 3.1529e-04
Epoch 11/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 46s 24ms/step - loss: 2.7611e-04
Epoch 12/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 46s 23ms/step - loss: 3.0707e-04
Epoch 13/20
1954/1954 ━━━━━━━━━━━━━━━━━━━━ 46s 24ms/step - loss: 3.4922e-04
Epoch 14/20
1954/1954 ━━━

In [91]:
new_data=yf.download('NVDA', start=end_date, end="2024-05-13")
new_scaled_data=scaler.transform(new_data['Close'].values.reshape(-1,1))

[*********************100%%**********************]  1 of 1 completed


In [92]:
test_data= new_scaled_data[-60:]
test_data = np.reshape(test_data, (1, 60, 1))

In [93]:
predicted_price = model.predict(test_data)
predicted_price = scaler.inverse_transform(predicted_price)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


In [95]:
print("Predicted price for May 13, 2024:", "${:.2f}".format(predicted_price[0][0]))

Predicted price for May 13, 2024: $76.79
